In [ ]:
import os
import csv
import cv2
import json
import math
from time import time
from itertools import product

In [ ]:
import vio_ort_exp as vio_ort

In [ ]:
# Инициализация параметров
odometry = vio_ort.VIO(lat0=54.889668, lon0=83.1258973333, alt0=0)
set_dir = '2024_12_15_15_31_8_num_3'
json_files = sorted([f for f in os.listdir(set_dir) if f.endswith('.json')])
start = 1000
count_json = 100

In [ ]:
# Значения для параметров
top_k_values = [512, 256]
detection_threshold_values = [0.05, 0.01]
max_iters_values = [None, 100, 300, 500]
rotation_methods = ["PIL", "CV2"]
trace_values = [8, 4]

In [ ]:
# Генерация всех комбинаций
parameters = [
    {'top_k': top_k, 'detection_threshold': detection_threshold, 'maxIters': max_iters, 'rotation': rotation, 'trace': trace}
    for top_k, detection_threshold, max_iters, rotation, trace in product(
        top_k_values, detection_threshold_values, max_iters_values, rotation_methods, trace_values
    )
]

In [ ]:
def run_vio(odometry, json_files, start, count_json, top_k, detection_threshold, maxIters, rotation, trace):
    """
    Выполняет обработку данных с использованием заданных параметров.
    """
    lat_VIO, lon_VIO, alt_VIO = [], [], []
    lat_GPS, lon_GPS, alt_GPS = [], [], []

    odometry._matcher.top_k = top_k
    odometry._matcher.detection_threshold = detection_threshold
    odometry.MAX_ITERS = maxIters
    odometry.ROTATION = rotation
    odometry.TRACE = trace

    for filename in json_files[start:start + count_json]:
        with open(f'{set_dir}/{filename}', 'r') as file:
            data = json.load(file)
            if 'GNRMC' in data and data['GNRMC']['status'] == 'A':
                img_path = os.path.join(set_dir, os.path.splitext(filename)[0] + '.jpg')
                image = cv2.imread(img_path)
                result_vio = odometry.add_trace_pt(image, data)

                lat_VIO.append(result_vio['lat'])
                lon_VIO.append(result_vio['lon'])
                alt_VIO.append(result_vio['alt'] * 1000)
                lat_GPS.append(data['GNRMC'].get('lat', 0.0))
                lon_GPS.append(data['GNRMC'].get('lon', 0.0))
                alt_GPS.append(data['GPS_RAW_INT']['alt'])

    return {
        'lat_VIO': lat_VIO,
        'lon_VIO': lon_VIO,
        'alt_VIO': alt_VIO,
        'lat_GPS': lat_GPS,
        'lon_GPS': lon_GPS,
        'alt_GPS': alt_GPS,
    }

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    """
    Функция для вычисления расстояния между двумя точками на поверхности Земли.
    """
    R = 6371000  # радиус Земли в метрах
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2) ** 2
    return R * 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

In [ ]:
def calculate_aggregated_metrics(results_optimized, results_original):
    """
    Функция для вычисления агрегированных метрик (среднее, максимальное, минимальное расстояние, RMSE).
    """
    metrics = {
        "Original VIO": [],
        "Optimized VIO": [],
    }

    for i in range(len(results_original['lat_GPS'])):
        gps_lat, gps_lon = results_original['lat_GPS'][i], results_original['lon_GPS'][i]
        org_lat, org_lon = results_original['lat_VIO'][i], results_original['lon_VIO'][i]
        opt_lat, opt_lon = results_optimized['lat_VIO'][i], results_optimized['lon_VIO'][i]

        metrics["Original VIO"].append(haversine(gps_lat, gps_lon, org_lat, org_lon))
        metrics["Optimized VIO"].append(haversine(gps_lat, gps_lon, opt_lat, opt_lon))

    aggregated = {
        "Metric": ["Mean Distance (m)", "Max Distance (m)", "Min Distance (m)", "RMSE (m)"],
    }

    for key, distances in metrics.items():
        mean_distance = sum(distances) / len(distances)
        max_distance = max(distances)
        min_distance = min(distances)
        rmse = math.sqrt(sum(d ** 2 for d in distances) / len(distances))

        aggregated[key] = [mean_distance, max_distance, min_distance, rmse]

    return aggregated

In [ ]:
def save_results_to_csv(results_all, filename):
    """
    Сохраняет результаты эксперимента в CSV файл.
    """
    with open(filename, mode='w', newline='') as f:
        writer = csv.writer(f)
        # Заголовок
        writer.writerow(["Top_k", "Detection Threshold", "Max Iters", "Rotation method", "Trace depth", "Time", "Mean Distance", "Max Distance", "RMSE"])

        # Данные
        for result in results_all:
            params = result['params']
            optimized_metrics = calculate_aggregated_metrics(result['results'], result['results'])['Optimized VIO']

            writer.writerow([
                params['top_k'], params['detection_threshold'], params['maxIters'], params['rotation'], params['trace'],
                result['time'],  # Используем 'time' как время выполнения
                optimized_metrics[0],  # Mean Distance
                optimized_metrics[1],  # Max Distance
                optimized_metrics[3],  # RMSE
            ])

In [ ]:
# Запуск экспериментов с разными параметрами с выводом прогресса
results_all = []
total = len(parameters)
for idx, params in enumerate(parameters):
    print(f"\nRunning experiment {idx+1}/{total} with parameters: {params}")
    start_time = time()
    results = run_vio(odometry, json_files, start, count_json, **params)
    elapsed_time = time() - start_time
    print(f"Experiment {idx+1} completed in {elapsed_time:.2f} seconds.")
    
    results_all.append({
        'params': params,
        'results': results,
        'time': elapsed_time
    })
    
    # Оценка оставшегося времени: если среднее время одного эксперимента равно elapsed_time,
    # то оставшееся время = (total - idx - 1) * elapsed_time
    remaining_time = (total - idx - 1) * elapsed_time
    print(f"Estimated remaining time: {remaining_time:.2f} seconds.")

# После цикла сохраняем результаты в CSV:
save_results_to_csv(results_all, "vio_results_comparison_1000.csv")

In [ ]:
# Сохраняем результаты в CSV
save_results_to_csv(results_all, "vio_results_comparison_100.csv")